In [ ]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/gdrive')

path1 = '/content/gdrive/MyDrive/Colab Notebooks/innopolis/train_dataset_train.csv'
path2 = '/content/gdrive/MyDrive/Colab Notebooks/innopolis/test_dataset_test.csv'

df_test = pd.read_csv(path2)
df_train = pd.read_csv(path1)



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
df_test['.geo'] = df_test['.geo'].str.replace(r"[^\d\.\,]", "", regex=True)
df_test['.geo'] = df_test['.geo'].str.replace(",", " ", regex=True)
df_test['temp'] = df_test['.geo'].str.count(".")
df_test['temp'] = df_test['temp']/100000
df_test['temp']

0       0.15019
1       0.00476
2       0.01346
3       0.00830
4       0.01453
         ...   
2066    0.00292
2067    0.00761
2068    0.00543
2069    0.15552
2070    0.00762
Name: temp, Length: 2071, dtype: float64

In [ ]:
df_train['.geo'] = df_train['.geo'].str.replace(r"[^\d\.\,]", "", regex=True)
df_train['.geo'] = df_train['.geo'].str.replace(",", " ", regex=True)
df_train['temp'] = df_train['.geo'].str.count(".")
df_train['temp'] = df_train['temp']/100000
df_train['temp']

0       0.01316
1       0.04873
2       0.00725
3       0.00584
4       0.01206
         ...   
4825    0.01893
4826    0.03354
4827    0.01680
4828    0.00477
4829    0.02075
Name: temp, Length: 4830, dtype: float64

In [ ]:
df_test_ = pd.read_csv(path2)
df_train_ = pd.read_csv(path1)

In [ ]:
df_train = df_train.replace({0:np.nan})
df_test = df_test.replace({0:np.nan})

In [ ]:
#df_test1 = df_test1.interpolate(asix = 1)

In [ ]:
df_test.fillna(df_test.groupby('area').transform("median"), inplace=True)
df_train.fillna(df_train.groupby('area').transform("median"), inplace=True)

In [ ]:
df_test['.geo'] = df_test_['.geo']
df_train['.geo'] = df_train_['.geo']

In [ ]:
def gs(js):
  x, y = [], []
  for dicr in js['geometries']:
    if dicr['type'] == 'Polygon':
      n = len(dicr['coordinates'][0])
      x_1 = sum(i[0] for i in dicr['coordinates'][0])/n
      y_1 = sum(i[1] for i in dicr['coordinates'][0])/n
      x.append(x_1)
      y.append(y_1)
  return sum(x) / len(x), sum(y) / len(y)

In [ ]:
def polygon(js):
  n = len(js['coordinates'][0])
  x_c = sum(x[0] for x in js['coordinates'][0])/n
  y_c = sum(x[1] for x in js['coordinates'][0])/n
  return x_c, y_c

In [ ]:
def mpolygon(js):
  x, y = [], []
  for lst in js['coordinates'][0]:
    x_1 = sum(x[0] for x in lst)/len(lst)
    y_1 = sum(y[1] for y in lst)/len(lst)
    x.append(x_1)
    y.append(y_1)
  return sum(x) / len(x), sum(y) / len(y)

In [ ]:
import json

In [ ]:
x, y = [], []
for val in df_test['.geo']:
  body = json.loads(val)
  if body['type'] == 'Polygon':
    x_c, y_c = polygon(body)
  elif body['type'] == 'GeometryCollection':
    x_c, y_c = gs(body)
  elif body['type'] == 'MultiPolygon':
    x_c, y_c = mpolygon(body)
  x.append(x_c)
  y.append(y_c)
df_test['X'] = x
df_test['Y'] = y 

In [ ]:
x, y = [], []
for val in df_train['.geo']:
  body = json.loads(val)
  if body['type'] == 'Polygon':
    x_c, y_c = polygon(body)
  elif body['type'] == 'GeometryCollection':
    x_c, y_c = gs(body)
  elif body['type'] == 'MultiPolygon':
    x_c, y_c = mpolygon(body)
  x.append(x_c)
  y.append(y_c)
df_train['X'] = x
df_train['Y'] = y 

In [ ]:
df_train = df_train.reindex(sorted(df_train.columns), axis=1)

In [ ]:
df_test = df_test.reindex(sorted(df_test.columns), axis=1)

In [ ]:
y_train = df_train['crop']
x_train = df_train.drop(columns= ['id', 'crop'])

y_test = df_test['id']
x_test = df_test.drop(['id'], axis=1)

In [ ]:
x_train = x_train.drop(['.geo'], axis=1)
x_test = x_test.drop(['.geo'], axis=1)

In [ ]:
x_train

,X,Y,area,nd_mean_2021-04-15,nd_mean_2021-04-16,nd_mean_2021-04-18,nd_mean_2021-04-19,nd_mean_2021-04-20,nd_mean_2021-04-22,nd_mean_2021-04-23,...,nd_mean_2021-07-31,nd_mean_2021-08-01,nd_mean_2021-08-07,nd_mean_2021-08-10,nd_mean_2021-08-11,nd_mean_2021-08-12,nd_mean_2021-08-13,nd_mean_2021-08-23,nd_mean_2021-08-27,temp
0,42.500303,51.412815,20,0.187150,0.072846,0.262300,0.261778,0.063134,0.062981,0.000000,...,0.000000,0.754859,0.000000,0.030883,0.000000,0.658016,0.126832,0.034858,0.614770,0.01316
1,50.265608,56.606045,45,0.245285,0.332928,0.349133,0.287182,0.064671,0.097138,0.024539,...,0.089721,0.706226,0.184231,0.062582,0.618475,0.601685,0.174914,0.000000,0.179612,0.04873
2,38.003231,53.837187,28,0.006391,0.013679,0.000000,0.007055,0.000000,0.120804,0.112471,...,0.411168,0.000000,0.357296,0.045370,0.172080,0.000000,0.090607,0.000000,0.054127,0.00725
3,44.727709,55.935536,19,0.293425,0.105976,0.318047,0.052500,0.153700,0.001917,0.045054,...,0.055983,0.152193,0.060656,0.352723,0.379196,0.327025,0.073492,0.061776,0.378900,0.00584
4,34.783534,52.907252,33,0.027983,0.130885,0.117227,0.063492,0.040565,0.024416,0.163886,...,0.611108,0.772962,0.000000,0.080592,0.000000,0.055220,0.157676,0.022251,0.012048,0.01206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4825,40.906039,50.899595,74,0.066154,0.028770,0.000000,0.038282,0.000000,0.255329,0.095812,...,0.379482,0.700605,0.645532,0.000000,0.059090,0.631144,0.042483,0.118901,0.022464,0.01893
4826,40.013739,51.804439,109,0.243867,0.035939,0.013403,0.030722,0.246513,0.033869,0.000000,...,0.800815,0.634428,0.783913,0.093728,0.000000,0.776900,0.055721,0.057945,0.044182,0.03354
4827,38.584332,53.193459,58,0.090026,0.191255,0.019897,0.063581,0.000000,0.140111,0.143922,...,0.740523,0.781434,0.783490,0.006700,0.132444,0.000000,0.007225,0.669367,0.145505,0.01680
4828,53.319120,55.183574,14,0.301663,0.291399,0.295089,0.221428,0.326494,0.007247,0.006888,...,0.235802,0.000000,0.079623,0.049245,0.506127,0.528256,0.031836,0.000000,0.453151,0.00477


In [ ]:
y_train = df_train['crop']

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf1 = RandomForestClassifier()
clf1.fit(x_train, y_train)

RandomForestClassifier()

In [ ]:
x_test.dtypes

area                    int64
nd_mean_2021-04-16    float64
nd_mean_2021-04-19    float64
nd_mean_2021-04-22    float64
nd_mean_2021-04-26    float64
                       ...   
nd_mean_2021-05-08    float64
nd_mean_2021-05-24    float64
temp                  float64
X                     float64
Y                     float64
Length: 75, dtype: object

In [ ]:
pre = clf1.predict(x_test)
pre

array([3, 2, 3, ..., 2, 6, 0])

In [ ]:
sub1 = pd.DataFrame({'id': y_test, 'crop': pre})
sub1.to_csv('result_crop60.csv', index=False)
sub1

,id,crop
0,611,3
1,6417,2
2,3352,3
3,4224,1
4,3102,6
...,...,...
2066,1743,5
2067,3968,3
2068,3809,2
2069,5485,6


In [ ]:
x_train

,area,nd_mean_2021-04-16,nd_mean_2021-04-19,nd_mean_2021-04-22,nd_mean_2021-04-26,nd_mean_2021-04-28,nd_mean_2021-05-02,nd_mean_2021-05-04,nd_mean_2021-05-07,nd_mean_2021-05-16,...,nd_mean_2021-06-05,nd_mean_2021-06-10,nd_mean_2021-07-05,nd_mean_2021-08-13,nd_mean_2021-08-27,nd_mean_2021-05-08,nd_mean_2021-05-24,X,Y,temp
0,20,0.072846,0.261778,0.062981,0.104442,0.021096,0.052202,0.158723,0.000000,0.484102,...,0.000000,0.000000,0.026784,0.126832,0.614770,0.008857,0.081498,42.500303,51.412815,0.01316
1,45,0.332928,0.287182,0.097138,0.152467,0.075484,0.000000,0.140326,0.163340,0.519456,...,0.000000,0.000000,0.111148,0.174914,0.179612,0.113071,0.046997,50.265608,56.606045,0.04873
2,28,0.013679,0.007055,0.120804,0.037839,0.259125,0.000000,0.037446,0.506516,0.000000,...,0.210496,0.130467,0.155225,0.090607,0.054127,0.007437,0.219614,38.003231,53.837187,0.00725
3,19,0.105976,0.052500,0.001917,0.011531,0.042316,0.000000,0.000000,0.043604,0.510271,...,0.132886,0.000000,0.570928,0.073492,0.378900,0.327677,0.586523,44.727709,55.935536,0.00584
4,33,0.130885,0.063492,0.024416,0.000000,0.150671,0.046717,0.254284,0.101479,0.000000,...,0.152003,0.073357,0.147842,0.157676,0.012048,0.054223,0.017539,34.783534,52.907252,0.01206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4825,74,0.028770,0.038282,0.255329,0.148166,0.225421,0.479483,0.071684,0.016856,0.407955,...,0.066882,0.139222,0.009041,0.042483,0.022464,0.017705,0.148440,40.906039,50.899595,0.01893
4826,109,0.035939,0.030722,0.033869,0.029646,0.318387,0.247905,0.265180,0.286298,0.578936,...,0.131357,0.000000,0.187236,0.055721,0.044182,0.048316,0.041440,40.013739,51.804439,0.03354
4827,58,0.191255,0.063581,0.140111,0.323984,0.000000,0.005056,0.092693,0.399439,0.006256,...,0.142930,0.000000,0.047572,0.007225,0.145505,0.051215,0.079730,38.584332,53.193459,0.01680
4828,14,0.291399,0.221428,0.007247,0.055074,0.057781,0.187749,0.112403,0.183555,0.504538,...,0.085861,0.631304,0.694524,0.031836,0.453151,0.000000,0.006495,53.319120,55.183574,0.00477


In [ ]:
df_test

,id,area,nd_mean_2021-04-16,nd_mean_2021-04-19,nd_mean_2021-04-22,nd_mean_2021-04-26,nd_mean_2021-04-28,nd_mean_2021-05-02,nd_mean_2021-05-04,nd_mean_2021-05-07,...,nd_mean_2021-06-05,nd_mean_2021-06-10,nd_mean_2021-07-05,nd_mean_2021-08-13,nd_mean_2021-08-27,nd_mean_2021-05-08,nd_mean_2021-05-24,X,Y,temp
0,611,26,0.196691,0.000000,0.259303,0.052522,0.133353,0.000000,0.058353,0.273228,...,0.188092,0.000000,0.200005,0.714292,0.084197,0.157733,0.197326,35.974008,52.385040,0.15019
1,6417,98,0.091192,0.000000,0.125823,0.000000,0.027636,0.433244,0.344155,0.461029,...,0.067910,0.084682,0.025694,0.000000,0.108945,0.141947,0.000000,39.517071,52.068139,0.00476
2,3352,21,0.000000,0.033368,0.285057,0.028275,0.089219,0.080082,0.220537,0.154620,...,0.044020,0.762353,0.167975,0.037838,0.158279,0.073814,0.185710,39.304960,49.945821,0.01346
3,4224,18,0.111243,0.201012,0.000000,0.000000,0.107940,0.000000,0.213204,0.031273,...,0.114008,0.059159,0.058423,0.108990,0.237742,0.000000,0.000000,33.085631,52.550573,0.00830
4,3102,53,0.000000,0.234954,0.000000,0.305964,0.268629,0.305554,0.131192,0.009491,...,0.209458,0.033658,0.000000,0.048622,0.121296,0.010354,0.000000,45.110874,55.373235,0.01453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2066,1743,18,0.000000,0.017866,0.212560,0.231169,0.012535,0.000000,0.103303,0.014683,...,0.018102,0.059161,0.804364,0.077803,0.112885,0.209859,0.269576,38.684136,54.015049,0.00292
2067,3968,56,0.010670,0.105135,0.170843,0.227927,0.000000,0.000000,0.000000,0.258106,...,0.109647,0.744611,0.554205,0.572680,0.511447,0.101374,0.119901,56.333597,53.491034,0.00761
2068,3809,92,0.131978,0.058492,0.607262,0.389112,0.061173,0.185756,0.000000,0.545505,...,0.000000,0.055619,0.051682,0.008488,0.076760,0.038590,0.086436,37.903133,53.139053,0.00543
2069,5485,53,0.044726,0.093328,0.188758,0.290000,0.178694,0.286592,0.041403,0.241813,...,0.053296,0.200393,0.000000,0.040434,0.037837,0.000000,0.165996,38.989850,51.863823,0.15552


In [ ]:
df_test.to_csv('df_test2.csv', index=False)
df_train.to_csv('df_train2.csv', index=False)

In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5, weights='uniform')
imputer.fit(x_train)

KNNImputer()

In [ ]:
x_train1 = imputer.transform(x_train)

In [ ]:
myImputer_data = pd.DataFrame(x_train1,columns = x_train.columns)

In [ ]:
myImputer_data

,area,nd_mean_2021-04-16,nd_mean_2021-04-19,nd_mean_2021-04-22,nd_mean_2021-04-26,nd_mean_2021-04-28,nd_mean_2021-05-02,nd_mean_2021-05-04,nd_mean_2021-05-07,nd_mean_2021-05-16,...,nd_mean_2021-06-05,nd_mean_2021-06-10,nd_mean_2021-07-05,nd_mean_2021-08-13,nd_mean_2021-08-27,nd_mean_2021-05-08,nd_mean_2021-05-24,temp,X,Y
0,20.0,0.072846,0.261778,0.062981,0.104442,0.021096,0.052202,0.158723,0.041219,0.484102,...,0.124326,0.125313,0.026784,0.126832,0.614770,0.008857,0.081498,0.01316,42.500303,51.412815
1,45.0,0.332928,0.287182,0.097138,0.152467,0.075484,0.177430,0.140326,0.163340,0.519456,...,0.200065,0.287425,0.111148,0.174914,0.179612,0.113071,0.046997,0.04873,50.265608,56.606045
2,28.0,0.013679,0.007055,0.120804,0.037839,0.259125,0.103707,0.037446,0.506516,0.353498,...,0.210496,0.130467,0.155225,0.090607,0.054127,0.007437,0.219614,0.00725,38.003231,53.837187
3,19.0,0.105976,0.052500,0.001917,0.011531,0.042316,0.122321,0.081126,0.043604,0.510271,...,0.132886,0.086913,0.570928,0.073492,0.378900,0.327677,0.586523,0.00584,44.727709,55.935536
4,33.0,0.130885,0.063492,0.024416,0.080858,0.150671,0.046717,0.254284,0.101479,0.439214,...,0.152003,0.073357,0.147842,0.157676,0.012048,0.054223,0.017539,0.01206,34.783534,52.907252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4825,74.0,0.028770,0.038282,0.255329,0.148166,0.225421,0.479483,0.071684,0.016856,0.407955,...,0.066882,0.139222,0.009041,0.042483,0.022464,0.017705,0.148440,0.01893,40.906039,50.899595
4826,109.0,0.035939,0.030722,0.033869,0.029646,0.318387,0.247905,0.265180,0.286298,0.578936,...,0.131357,0.067455,0.187236,0.055721,0.044182,0.048316,0.041440,0.03354,40.013739,51.804439
4827,58.0,0.191255,0.063581,0.140111,0.323984,0.096556,0.005056,0.092693,0.399439,0.006256,...,0.142930,0.101047,0.047572,0.007225,0.145505,0.051215,0.079730,0.01680,38.584332,53.193459
4828,14.0,0.291399,0.221428,0.007247,0.055074,0.057781,0.187749,0.112403,0.183555,0.504538,...,0.085861,0.631304,0.694524,0.031836,0.453151,0.045079,0.006495,0.00477,53.319120,55.183574


In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5, weights='uniform')
imputer.fit(x_test)

KNNImputer()

In [ ]:
x_test1 = imputer.transform(x_test)

In [ ]:
myImputer_data_ts = pd.DataFrame(x_test1,columns = x_test.columns)

In [ ]:
myImputer_data_ts

,area,nd_mean_2021-04-16,nd_mean_2021-04-19,nd_mean_2021-04-22,nd_mean_2021-04-26,nd_mean_2021-04-28,nd_mean_2021-05-02,nd_mean_2021-05-04,nd_mean_2021-05-07,nd_mean_2021-05-16,...,nd_mean_2021-06-05,nd_mean_2021-06-10,nd_mean_2021-07-05,nd_mean_2021-08-13,nd_mean_2021-08-27,nd_mean_2021-05-08,nd_mean_2021-05-24,temp,X,Y
0,26.0,0.196691,0.064780,0.259303,0.052522,0.133353,0.141064,0.058353,0.273228,0.944027,...,0.188092,0.353347,0.200005,0.714292,0.084197,0.157733,0.197326,0.15019,35.974008,52.385040
1,98.0,0.091192,0.164154,0.125823,0.320995,0.027636,0.433244,0.344155,0.461029,0.429587,...,0.067910,0.084682,0.025694,0.084338,0.108945,0.141947,0.149115,0.00476,39.517071,52.068139
2,21.0,0.175323,0.033368,0.285057,0.028275,0.089219,0.080082,0.220537,0.154620,0.612726,...,0.044020,0.762353,0.167975,0.037838,0.158279,0.073814,0.185710,0.01346,39.304960,49.945821
3,18.0,0.111243,0.201012,0.453035,0.107636,0.107940,0.057303,0.213204,0.031273,0.434872,...,0.114008,0.059159,0.058423,0.108990,0.237742,0.082367,0.078110,0.00830,33.085631,52.550573
4,53.0,0.144268,0.234954,0.078573,0.305964,0.268629,0.305554,0.131192,0.009491,0.480222,...,0.209458,0.033658,0.263500,0.048622,0.121296,0.010354,0.297788,0.01453,45.110874,55.373235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2066,18.0,0.120758,0.017866,0.212560,0.231169,0.012535,0.097705,0.103303,0.014683,0.344103,...,0.018102,0.059161,0.804364,0.077803,0.112885,0.209859,0.269576,0.00292,38.684136,54.015049
2067,56.0,0.010670,0.105135,0.170843,0.227927,0.095751,0.140273,0.097191,0.258106,0.575754,...,0.109647,0.744611,0.554205,0.572680,0.511447,0.101374,0.119901,0.00761,56.333597,53.491034
2068,92.0,0.131978,0.058492,0.607262,0.389112,0.061173,0.185756,0.120462,0.545505,0.260217,...,0.085916,0.055619,0.051682,0.008488,0.076760,0.038590,0.086436,0.00543,37.903133,53.139053
2069,53.0,0.044726,0.093328,0.188758,0.290000,0.178694,0.286592,0.041403,0.241813,0.499471,...,0.053296,0.200393,0.136789,0.040434,0.037837,0.119188,0.165996,0.15552,38.989850,51.863823


In [ ]:
x_test = myImputer_data_ts
x_train = myImputer_data

In [ ]:
x_test

In [ ]:
x_train_new = x_train[np.isfinite(x_train).all(1)]

In [ ]:
x_train_new = x_train_new.iloc[0:27785]

In [ ]:
x_train_new

,area,nd_mean_2021-04-16,nd_mean_2021-04-19,nd_mean_2021-04-22,nd_mean_2021-04-26,nd_mean_2021-04-28,nd_mean_2021-05-02,nd_mean_2021-05-04,nd_mean_2021-05-07,nd_mean_2021-05-16,...,nd_mean_2021-06-05,nd_mean_2021-06-10,nd_mean_2021-07-05,nd_mean_2021-08-13,nd_mean_2021-08-27,nd_mean_2021-05-08,nd_mean_2021-05-24,X,Y,temp
0,20.0,0.072846,0.261778,0.062981,0.104442,0.021096,0.052202,0.158723,0.041219,0.484102,...,0.124326,0.125313,0.026784,0.126832,0.614770,0.008857,0.081498,42.500303,51.412815,0.01316
1,45.0,0.332928,0.287182,0.097138,0.152467,0.075484,0.177430,0.140326,0.163340,0.519456,...,0.200065,0.287425,0.111148,0.174914,0.179612,0.113071,0.046997,50.265608,56.606045,0.04873
2,28.0,0.013679,0.007055,0.120804,0.037839,0.259125,0.103707,0.037446,0.506516,0.353498,...,0.210496,0.130467,0.155225,0.090607,0.054127,0.007437,0.219614,38.003231,53.837187,0.00725
3,19.0,0.105976,0.052500,0.001917,0.011531,0.042316,0.122321,0.081126,0.043604,0.510271,...,0.132886,0.086913,0.570928,0.073492,0.378900,0.327677,0.586523,44.727709,55.935536,0.00584
4,33.0,0.130885,0.063492,0.024416,0.080858,0.150671,0.046717,0.254284,0.101479,0.439214,...,0.152003,0.073357,0.147842,0.157676,0.012048,0.054223,0.017539,34.783534,52.907252,0.01206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4825,74.0,0.028770,0.038282,0.255329,0.148166,0.225421,0.479483,0.071684,0.016856,0.407955,...,0.066882,0.139222,0.009041,0.042483,0.022464,0.017705,0.148440,40.906039,50.899595,0.01893
4826,109.0,0.035939,0.030722,0.033869,0.029646,0.318387,0.247905,0.265180,0.286298,0.578936,...,0.131357,0.067455,0.187236,0.055721,0.044182,0.048316,0.041440,40.013739,51.804439,0.03354
4827,58.0,0.191255,0.063581,0.140111,0.323984,0.096556,0.005056,0.092693,0.399439,0.006256,...,0.142930,0.101047,0.047572,0.007225,0.145505,0.051215,0.079730,38.584332,53.193459,0.01680
4828,14.0,0.291399,0.221428,0.007247,0.055074,0.057781,0.187749,0.112403,0.183555,0.504538,...,0.085861,0.631304,0.694524,0.031836,0.453151,0.045079,0.006495,53.319120,55.183574,0.00477


In [ ]:
x_train_new = x_train_new.fillna(0)

In [ ]:
y_train = y_train.fillna(0)

In [ ]:
x_test = x_test.drop(['temp'], axis = 1)
x_train = x_train.drop(['temp'], axis = 1)

In [ ]:
x_train

,area,nd_mean_2021-04-16,nd_mean_2021-04-19,nd_mean_2021-04-22,nd_mean_2021-04-26,nd_mean_2021-04-28,nd_mean_2021-05-02,nd_mean_2021-05-04,nd_mean_2021-05-07,nd_mean_2021-05-16,...,nd_mean_2021-06-04,nd_mean_2021-06-05,nd_mean_2021-06-10,nd_mean_2021-07-05,nd_mean_2021-08-13,nd_mean_2021-08-27,nd_mean_2021-05-08,nd_mean_2021-05-24,X,Y
0,20.0,0.072846,0.261778,0.062981,0.104442,0.021096,0.052202,0.158723,0.041219,0.484102,...,0.118854,0.124326,0.125313,0.026784,0.126832,0.614770,0.008857,0.081498,42.500303,51.412815
1,45.0,0.332928,0.287182,0.097138,0.152467,0.075484,0.177430,0.140326,0.163340,0.519456,...,0.208268,0.200065,0.287425,0.111148,0.174914,0.179612,0.113071,0.046997,50.265608,56.606045
2,28.0,0.013679,0.007055,0.120804,0.037839,0.259125,0.103707,0.037446,0.506516,0.353498,...,0.079094,0.210496,0.130467,0.155225,0.090607,0.054127,0.007437,0.219614,38.003231,53.837187
3,19.0,0.105976,0.052500,0.001917,0.011531,0.042316,0.122321,0.081126,0.043604,0.510271,...,0.642226,0.132886,0.086913,0.570928,0.073492,0.378900,0.327677,0.586523,44.727709,55.935536
4,33.0,0.130885,0.063492,0.024416,0.080858,0.150671,0.046717,0.254284,0.101479,0.439214,...,0.099058,0.152003,0.073357,0.147842,0.157676,0.012048,0.054223,0.017539,34.783534,52.907252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4825,74.0,0.028770,0.038282,0.255329,0.148166,0.225421,0.479483,0.071684,0.016856,0.407955,...,0.031995,0.066882,0.139222,0.009041,0.042483,0.022464,0.017705,0.148440,40.906039,50.899595
4826,109.0,0.035939,0.030722,0.033869,0.029646,0.318387,0.247905,0.265180,0.286298,0.578936,...,0.255964,0.131357,0.067455,0.187236,0.055721,0.044182,0.048316,0.041440,40.013739,51.804439
4827,58.0,0.191255,0.063581,0.140111,0.323984,0.096556,0.005056,0.092693,0.399439,0.006256,...,0.187352,0.142930,0.101047,0.047572,0.007225,0.145505,0.051215,0.079730,38.584332,53.193459
4828,14.0,0.291399,0.221428,0.007247,0.055074,0.057781,0.187749,0.112403,0.183555,0.504538,...,0.644357,0.085861,0.631304,0.694524,0.031836,0.453151,0.045079,0.006495,53.319120,55.183574


In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf2 = RandomForestClassifier()
clf2.fit(x_train, y_train)

RandomForestClassifier()

In [ ]:
pre3 = clf2.predict(x_test)
sub1 = pd.DataFrame({'id': y_test, 'crop': pre3})
sub1['crop'] = sub1['crop'].astype("int")
sub1.to_csv('result3e_crop46.csv', index=False)
sub1

,id,crop
0,611,3
1,6417,2
2,3352,4
3,4224,1
4,3102,6
...,...,...
2066,1743,5
2067,3968,3
2068,3809,2
2069,5485,6


In [ ]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.3 MB/s 


In [ ]:
df_train.columns #упорядочить столбцы

Index(['id', 'area', 'nd_mean_2021-04-16', 'nd_mean_2021-04-19',
       'nd_mean_2021-04-22', 'nd_mean_2021-04-26', 'nd_mean_2021-04-28',
       'nd_mean_2021-05-02', 'nd_mean_2021-05-04', 'nd_mean_2021-05-07',
       'nd_mean_2021-05-16', 'nd_mean_2021-05-17', 'nd_mean_2021-05-21',
       'nd_mean_2021-05-26', 'nd_mean_2021-05-27', 'nd_mean_2021-05-29',
       'nd_mean_2021-06-06', 'nd_mean_2021-06-09', 'nd_mean_2021-06-13',
       'nd_mean_2021-06-16', 'nd_mean_2021-06-18', 'nd_mean_2021-06-19',
       'nd_mean_2021-06-20', 'nd_mean_2021-06-22', 'nd_mean_2021-06-25',
       'nd_mean_2021-06-27', 'nd_mean_2021-07-07', 'nd_mean_2021-07-08',
       'nd_mean_2021-07-09', 'nd_mean_2021-07-13', 'nd_mean_2021-07-15',
       'nd_mean_2021-07-17', 'nd_mean_2021-07-20', 'nd_mean_2021-07-26',
       'nd_mean_2021-07-27', 'nd_mean_2021-07-29', 'nd_mean_2021-07-31',
       'nd_mean_2021-08-07', 'nd_mean_2021-08-12', '.geo',
       'nd_mean_2021-04-15', 'nd_mean_2021-04-18', 'nd_mean_2021-04-25',


In [ ]:
from catboost import CatBoostClassifier

In [ ]:
cat = CatBoostClassifier()

In [ ]:
x_train

,area,nd_mean_2021-04-16,nd_mean_2021-04-19,nd_mean_2021-04-22,nd_mean_2021-04-26,nd_mean_2021-04-28,nd_mean_2021-05-02,nd_mean_2021-05-04,nd_mean_2021-05-07,nd_mean_2021-05-16,...,nd_mean_2021-06-05,nd_mean_2021-06-10,nd_mean_2021-07-05,nd_mean_2021-08-13,nd_mean_2021-08-27,nd_mean_2021-05-08,nd_mean_2021-05-24,temp,X,Y
0,20.0,0.072846,0.261778,0.062981,0.104442,0.021096,0.052202,0.158723,0.041219,0.484102,...,0.124326,0.125313,0.026784,0.126832,0.614770,0.008857,0.081498,0.01316,42.500303,51.412815
1,45.0,0.332928,0.287182,0.097138,0.152467,0.075484,0.177430,0.140326,0.163340,0.519456,...,0.200065,0.287425,0.111148,0.174914,0.179612,0.113071,0.046997,0.04873,50.265608,56.606045
2,28.0,0.013679,0.007055,0.120804,0.037839,0.259125,0.103707,0.037446,0.506516,0.353498,...,0.210496,0.130467,0.155225,0.090607,0.054127,0.007437,0.219614,0.00725,38.003231,53.837187
3,19.0,0.105976,0.052500,0.001917,0.011531,0.042316,0.122321,0.081126,0.043604,0.510271,...,0.132886,0.086913,0.570928,0.073492,0.378900,0.327677,0.586523,0.00584,44.727709,55.935536
4,33.0,0.130885,0.063492,0.024416,0.080858,0.150671,0.046717,0.254284,0.101479,0.439214,...,0.152003,0.073357,0.147842,0.157676,0.012048,0.054223,0.017539,0.01206,34.783534,52.907252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4825,74.0,0.028770,0.038282,0.255329,0.148166,0.225421,0.479483,0.071684,0.016856,0.407955,...,0.066882,0.139222,0.009041,0.042483,0.022464,0.017705,0.148440,0.01893,40.906039,50.899595
4826,109.0,0.035939,0.030722,0.033869,0.029646,0.318387,0.247905,0.265180,0.286298,0.578936,...,0.131357,0.067455,0.187236,0.055721,0.044182,0.048316,0.041440,0.03354,40.013739,51.804439
4827,58.0,0.191255,0.063581,0.140111,0.323984,0.096556,0.005056,0.092693,0.399439,0.006256,...,0.142930,0.101047,0.047572,0.007225,0.145505,0.051215,0.079730,0.01680,38.584332,53.193459
4828,14.0,0.291399,0.221428,0.007247,0.055074,0.057781,0.187749,0.112403,0.183555,0.504538,...,0.085861,0.631304,0.694524,0.031836,0.453151,0.045079,0.006495,0.00477,53.319120,55.183574


In [ ]:
y_train = df_train['crop']

In [ ]:
##############

In [ ]:
df_train_new = pd.read_csv(path1)

In [ ]:
y_train = df_train_new['crop']

In [ ]:
y_train

0       3.0
1       4.0
2       2.0
3       5.0
4       4.0
       ... 
4825    6.0
4826    6.0
4827    6.0
4828    5.0
4829    6.0
Name: crop, Length: 4830, dtype: float64

In [ ]:
y_train.isna().sum()

0

In [ ]:
cat.fit(x_train,y_train,verbose=False, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [ ]:
pre21 =  np.reshape(pre21, 2071)

In [ ]:
pre21 == pre2

array([ True,  True,  True, ...,  True,  True,  True])

In [ ]:
pre3321 = cat.predict(x_test)
sub1 = pd.DataFrame({'id': y_test, 'crop': pre21})
sub1.to_csv('result3e_crop54.csv', index=False)
sub1

,id,crop
0,611,3
1,6417,2
2,3352,3
3,4224,1
4,3102,6
...,...,...
2066,1743,5
2067,3968,3
2068,3809,2
2069,5485,6
